In [33]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from tensorflow.keras.datasets import imdb
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import sequence
seed = 123

# Import des données et pré-processing

In [34]:
NUM_WORDS = 5000
max_review_length = 100
INDEX_FROM = 3

# --- Import the IMDB data and only consider the ``top_words``` most used words
np.load.__defaults__=(None, True, True, 'ASCII')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)
np.load.__defaults__=(None, False, True, 'ASCII')

word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in X_train[1000] ))

<START> although i had seen <UNK> in a theater way back in <UNK> i couldn't remember anything of the plot except for vague images of kurt thomas running and fighting against a backdrop of stone walls and disappointment regarding the ending br br after reading some of the other reviews i picked up a copy of the newly released dvd to once again enter the world of <UNK> br br it turns out this is one of those films produced during the <UNK> that would go directly to video today the film stars <UNK> <UNK> kurt thomas as jonathan <UNK> <UNK> out of the blue to <UNK> the nation of <UNK> to enter and hopefully win the game a <UNK> <UNK> <UNK> by the khan who <UNK> his people by yelling what sounds like <UNK> power the goal of the mission involves the star wars defense system jonathan is trained in the martial arts by princess <UNK> who never speaks or leaves the house once trained tries to blend in with the <UNK> by wearing a bright red <UNK> with <UNK> of blue and white needless to say <UNK>

In [35]:
# --- truncate and pad input sequences
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length, padding='post', truncating='post', value=0)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length, padding='post', truncating='post', value=0)

print("len(X_train[0]):", len(X_train[0]))
print("len(X_train[1]):", len(X_train[1]))
print("X_train[0]:", X_train[0])

len(X_train[0]): 100
len(X_train[1]): 100
X_train[0]: [   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
    2   16]


# LSTM

In [36]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, topics, model_length):
        self.topics=topics
        self.model_length=model_length

    def __len__(self):
        return len(self.topics)-self.model_length

    def __getitem__(self, index):
        input_sequence=torch.tensor(self.topics[index:index+self.model_length, :])
        target_sequence=torch.tensor(self.topics[index+1:index+self.model_length+1, :])

        return input_sequence, target_sequence

In [37]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model_length):
        super(LSTM, self).__init__()
        self.input_size = input_size #dimension d'entrée (NUM_TOPICS)
        self.hidden_size = hidden_size #nombre de neurones de la couche cachée
        self.output_size = output_size #dimension d'outputs (NUM_TOPICS)
        self.model_length=model_length

        self.lstm = nn.LSTM(self.input_size, self.hidden_size, batch_first=True)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        # self.softmax = nn.Softmax(dim=1) 

    def forward(self, x, prev_state):
        output, state = self.lstm(x, prev_state)
        output=self.fc(output)
        probabilities = F.softmax(output[:, -1, :], dim=1)
        return probabilities, state
    
    def init_state(self):
        return (torch.zeros(1, 1, self.hidden_size), #(NUM_LAYERS, BATCH SIZE, NUM_NEURONES)
                torch.zeros(1, 1, self.hidden_size))
    
    def train_model(self, dataset, optimizer, criterion):
        state_h, state_c = self.init_state()
        self.train()
        for t, (x, y) in enumerate(dataset):
            optimizer.zero_grad()
            softmax , (state_h, state_c) = self(x, (state_h, state_c)) #softmax= p(z_{t+1}|z_1:t)
            loss = criterion(softmax, y[:, -1, :])
            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()
            print({'loss': loss.item() })
    
    def predict_next_probability(self, input_sequence):
        state_h, state_c = self.init_state()

        # Forward pass jusqu'à t-1
        for t in range(len(input_sequence)):
            input_t = input_sequence[t].unsqueeze(0).unsqueeze(0)
            _, (state_h, state_c) = self(input_t, (state_h, state_c))

        # Obtenez les probabilités pour x_t
        input_t = input_sequence[-1].unsqueeze(0).unsqueeze(0)
        probabilities, _ = self(input_t, (state_h, state_c))

        return probabilities
    
    def sample_next_z(self, input_sequence):
        proba = self.predict_next_probability(input_sequence)
        return torch.multinomial(proba, 1).item()

## SSM

In [115]:
class SSM:
    def __init__(self, num_words, num_topics, T):
        self.num_words = num_words
        self.num_topics = num_topics
        self.T = T
        self.phi = np.random.randn(T, num_words, num_topics) * 0.01
        self.phi = np.exp(self.phi - np.max(self.phi, axis=1, keepdims=True))
        self.phi = self.phi / np.sum(self.phi, axis=1, keepdims=True)


    def compute_MLE_SSM(self, ech_x, ech_z):
        def compute_MLE_SSM_time_t_zt(t, z_t, ech_x, ech_z, num_words):
            phi = np.zeros(num_words)
            list_probas = []
            for j in range(1,num_words+1):
                num = len(np.where((ech_x[:,t-1]==j)&(ech_z[:,t-1]==z_t))[0])/ech_x.shape[0]
                phi[j-1] = num
                list_probas.append(num)

            denom = np.sum(np.array(list_probas))
            phi = phi / (denom + 1e-6)
            return phi
        
        def compute_MLE_SSM_time_t(t, ech_x, ech_z, num_words, num_topics):
            phi = np.zeros((num_words, num_topics))
            for k in range(1, num_topics+1):
                z_t = k
                phi_zt = compute_MLE_SSM_time_t_zt(t=t, z_t=z_t, ech_x=ech_x, ech_z=ech_z, num_words=num_words)
                phi[:,k-1] = phi_zt
            return phi
        
        for t in range(self.T):
            self.phi[t] = compute_MLE_SSM_time_t(t, ech_x, ech_z, self.num_words, self.num_topics)
    
    def predict_proba(self, t, z_t):
        return self.phi[t-1][:,z_t-1]
    
    def sample_xt(self, t, z_t):
        proba = self.predict_proba(t, z_t)
        sampled_xt = np.random.choice(len(proba), p=proba)
        return sampled_xt
        

## Particle Gibbs

In [116]:
def compute_alpha_unnormalized(t, z_1_t_minus_1, num_topics, num_voc, lstm, ssm):
    z_1_t_minus_1=z_1_t_minus_1-1
    z_one_hot = to_categorical(z_1_t_minus_1, num_classes=num_topics)
    z_one_hot_tensor = torch.FloatTensor(z_one_hot)
    softmax = lstm.predict_next_probability(z_one_hot_tensor).detach().numpy()[0]
    phi_t = ssm.phi[t-1]
    alpha = np.array([np.dot(softmax, phi_t[j,:]) for j in range(num_voc)])
    return alpha

def compute_alpha_normalized(t, z_1_t_minus_1, num_topics, num_voc, lstm, ssm):
    num = compute_alpha_unnormalized(t, z_1_t_minus_1, num_topics, num_voc, lstm, ssm)
    denom = np.sum(num)
    return num/(denom+1e-6)

def compute_gamma_unnormalized(t, xt, z_1_t_minus_1, num_topics, lstm, ssm):
    z_1_t_minus_1=z_1_t_minus_1-1
    z_one_hot = to_categorical(z_1_t_minus_1, num_classes=num_topics)
    z_one_hot_tensor = torch.FloatTensor(z_one_hot)
    softmax = lstm.predict_next_probability(z_one_hot_tensor).detach().numpy()[0]
    phi_t = ssm.phi[t-1]
    phi_xt = phi_t[xt-1, :]
    return np.multiply(softmax, phi_xt)

def compute_gamma_normalized(t, xt, z_1_t_minus_1, num_topics, lstm, ssm):
    num = compute_gamma_unnormalized(t, xt, z_1_t_minus_1, num_topics, lstm, ssm)
    denom = np.sum(num)
    return num/(denom+1e-6)

In [117]:
def particle_gibbs(P, num_topics, num_words, T, lstm_model, ssm_model, x, previous_z_1_T_star):
    ##Init
    Z_matrix=np.zeros((P, T+1))
    alpha_matrix=np.zeros((P,T+1))
    ancestor_matrix=np.ones((P,T+1))
    ##t=0
    z_0 = np.random.choice(a=range(1,num_topics+1), size=P)
    alpha_0 = np.repeat(1/P, P)
    Z_matrix[:,0] = z_0
    alpha_matrix[:,0] = alpha_0

    #z[k:n]: du k-ème au n-1 ème
    for t in range(1,T+1):
        a_t_minus_1 = 1 #ok
        z_1_t = previous_z_1_T_star[:t] #ok
        ancestor_matrix[0,t-1] = a_t_minus_1 #ok
        Z_matrix[0, 1:t+1] = z_1_t #ok

        for p in range(2,P+1):
            alpha_t_minus_1_p=alpha_matrix[:,t-1]
            #a_t_minus_1_p = np.random.choice(a=range(1,P+1), p=alpha_t_minus_1_p, size=1)[0] #ok
            a_t_minus_1_p = np.argmax(alpha_t_minus_1_p)+1
            ancestor_matrix[p-1, t-1] = a_t_minus_1_p #ok
            if t ==1:
                z_1_t_minus_1_a_t_minus_1_p = Z_matrix[int(a_t_minus_1_p)-1, 0] #ok
                z_1_t_minus_1_a_t_minus_1_p = np.array([z_1_t_minus_1_a_t_minus_1_p]) #ok
                gamma_t_p = compute_gamma_normalized(t=t,
                                                xt=x[t-1], 
                                                z_1_t_minus_1=z_1_t_minus_1_a_t_minus_1_p,
                                                num_topics = num_topics,
                                                lstm = lstm_model,
                                                ssm = ssm_model)
                z_t_p = np.argmax(gamma_t_p)+1
                z_1_t_p = z_t_p
            else:
                z_1_t_minus_1_a_t_minus_1_p = Z_matrix[int(a_t_minus_1_p)-1, 1:(t-1)+1] #ok
                gamma_t_p = compute_gamma_normalized(t=t,
                                                xt=x[t-1], 
                                                z_1_t_minus_1=z_1_t_minus_1_a_t_minus_1_p,
                                                num_topics = num_topics,
                                                lstm = lstm_model,
                                                ssm = ssm_model)
                z_t_p = np.argmax(gamma_t_p)+1
                #z_t_p = np.random.choice(a=range(1, NUM_TOPICS+1), p=gamma_t_p, size=1)[0]
                z_1_t_p = np.append(z_1_t_minus_1_a_t_minus_1_p, z_t_p)
            
            Z_matrix[p-1, 1:t+1] = z_1_t_p

        
        for p in range(1, P+1):
            a_t_minus_1_p = ancestor_matrix[p-1, t-1]
            if t ==1:
                z_1_t_minus_1_a_t_minus_1_p = Z_matrix[int(a_t_minus_1_p)-1, 0] #ok
                z_1_t_minus_1_a_t_minus_1_p = np.array([z_1_t_minus_1_a_t_minus_1_p]) #ok
            else:
                z_1_t_minus_1_a_t_minus_1_p = Z_matrix[int(a_t_minus_1_p)-1, 1:(t-1)+1]
            
            alpha_t_p = compute_alpha_normalized(t=t,
                                                z_1_t_minus_1=z_1_t_minus_1_a_t_minus_1_p,
                                                num_topics=num_topics,
                                                num_voc=num_words,
                                                lstm=lstm_model,
                                                ssm=ssm_model)
            alpha_t_p = alpha_t_p[x[t-1]-1]
            alpha_matrix[p-1,t] = alpha_t_p
    alpha_T=alpha_matrix[:,-1]
    alpha_T = alpha_T / np.sum(alpha_T+1e-6)
    r = np.argmax(alpha_T)+1
    a_T_r = ancestor_matrix[int(r)-1, -1]
    z_1_T = Z_matrix[int(a_T_r)-1, 1:]
    return z_1_T

## EM inference

In [118]:
HIDDEN_SIZE=64
SEQUENCE_LENGTH = 100
MODEL_LENGTH = 10
NUM_TOPICS = 50
NUM_WORDS = 5000
N_ITER_EM = 1
NUM_PARTICULES = 10

lstm_model=LSTM(input_size=NUM_TOPICS, hidden_size=HIDDEN_SIZE, output_size=NUM_TOPICS, model_length=MODEL_LENGTH)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.001)

ssm_model = SSM(num_words=NUM_WORDS, num_topics=NUM_TOPICS, T=SEQUENCE_LENGTH)

In [120]:
ssm_model.phi.shape

(100, 5000, 50)

In [125]:
phi = torch.randn((100, 5000, 50), requires_grad=True) * 0.01
phi
# Normalisation softmax
phi = F.softmax(phi, dim=1)
torch.sum(phi[0], axis=0)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)

In [77]:
for i in range(X_train.shape[0]):
    print("i : {}".format(i))
    x = X_train[i]
    previous_z_1_T_star = np.random.choice(a=range(1,NUM_TOPICS+1), size=SEQUENCE_LENGTH)
    for iter in range(N_ITER_EM):
        z_star = particle_gibbs(NUM_PARTICULES, NUM_TOPICS, NUM_WORDS, SEQUENCE_LENGTH, lstm_model, ssm_model, x, previous_z_1_T_star)
        z_star_one_hot = to_categorical(z_star-1, num_classes=NUM_TOPICS)
        dataset=Dataset(topics=z_star_one_hot, model_length=MODEL_LENGTH)
        dataloader = DataLoader(dataset, batch_size=1)
        lstm_model.train_model(dataloader, optimizer, criterion)
        ech_x = np.expand_dims(x, axis=0)
        ech_z = np.expand_dims(z_star, axis=0)
        ssm_model.compute_MLE_SSM(ech_x, ech_z)
        previous_z_1_T_star = z_star_one_hot

i : 0
{'loss': 3.914184093475342}
{'loss': 3.9120841026306152}
{'loss': 3.910663366317749}
{'loss': 3.911644220352173}
{'loss': 3.9115757942199707}
{'loss': 3.9136569499969482}
{'loss': 3.9102933406829834}
{'loss': 3.911639928817749}
{'loss': 3.9094855785369873}
{'loss': 3.914017915725708}
{'loss': 3.909961462020874}
{'loss': 3.9099209308624268}
{'loss': 3.9139115810394287}
{'loss': 3.9113500118255615}
{'loss': 3.9088540077209473}
{'loss': 3.912597179412842}
{'loss': 3.9115169048309326}
{'loss': 3.9126017093658447}
{'loss': 3.915341377258301}
{'loss': 3.911907911300659}
{'loss': 3.9115161895751953}
{'loss': 3.9141688346862793}
{'loss': 3.913803815841675}
{'loss': 3.913529396057129}
{'loss': 3.911194086074829}
{'loss': 3.909653425216675}
{'loss': 3.9093902111053467}
{'loss': 3.910464286804199}
{'loss': 3.9083778858184814}
{'loss': 3.9146018028259277}
{'loss': 3.911693572998047}
{'loss': 3.9101462364196777}
{'loss': 3.914497137069702}
{'loss': 3.9116764068603516}
{'loss': 3.9114274978637

KeyboardInterrupt: 

In [ ]:
# ##Initialisation
# P=50
# NUM_TOPICS=50
# NUM_WORDS=5000
# T=100

# lstm = lstm_model
# ssm = ssm_model

# Z_matrix=np.zeros((P, T+1))
# alpha_matrix=np.zeros((P,T+1))
# ancestor_matrix=np.zeros((P,T+1))


# z_1_T_star = np.random.choice(a=range(1,NUM_TOPICS+1), size=T)
# x = X_train[0]
# x

In [ ]:
# ##t=0
# z_0 = np.random.choice(a=range(1,NUM_TOPICS+1), size=P)
# alpha_0 = np.repeat(1/P, P)
# Z_matrix[:,0] = z_0
# alpha_matrix[:,0] = alpha_0

# #z[k:n]: du k-ème au n-1 ème
# for t in range(1,T+1):
#     print(t)
#     a_t_minus_1 = 1 #ok
#     z_1_t = z_1_T_star[:t] #ok
#     ancestor_matrix[0,t-1] = a_t_minus_1 #ok
#     Z_matrix[0, 1:t+1] = z_1_t #ok

#     for p in range(2,P+1):
#         alpha_t_minus_1_p=alpha_matrix[:,t-1]
#         #a_t_minus_1_p = np.random.choice(a=range(1,P+1), p=alpha_t_minus_1_p, size=1)[0] #ok
#         a_t_minus_1_p = np.argmax(alpha_t_minus_1_p)+1
#         ancestor_matrix[p-1, t-1] = a_t_minus_1_p #ok
#         if t ==1:
#             z_1_t_minus_1_a_t_minus_1_p = Z_matrix[int(a_t_minus_1_p)-1, 0] #ok
#             z_1_t_minus_1_a_t_minus_1_p = np.array([z_1_t_minus_1_a_t_minus_1_p]) #ok
#             gamma_t_p = compute_gamma_normalized(t=t,
#                                              xt=x[t-1], 
#                                              z_1_t_minus_1=z_1_t_minus_1_a_t_minus_1_p,
#                                              num_topics = NUM_TOPICS,
#                                              lstm = lstm,
#                                              ssm = ssm)
#             z_t_p = np.argmax(gamma_t_p)+1
#             z_1_t_p = z_t_p
#         else:
#             z_1_t_minus_1_a_t_minus_1_p = Z_matrix[int(a_t_minus_1_p)-1, 1:(t-1)+1] #ok
#             gamma_t_p = compute_gamma_normalized(t=t,
#                                              xt=x[t-1], 
#                                              z_1_t_minus_1=z_1_t_minus_1_a_t_minus_1_p,
#                                              num_topics = NUM_TOPICS,
#                                              lstm = lstm,
#                                              ssm = ssm)
#             z_t_p = np.argmax(gamma_t_p)+1
#             #z_t_p = np.random.choice(a=range(1, NUM_TOPICS+1), p=gamma_t_p, size=1)[0]
#             z_1_t_p = np.append(z_1_t_minus_1_a_t_minus_1_p, z_t_p)
        
#         Z_matrix[p-1, 1:t+1] = z_1_t_p

    
#     for p in range(1, P+1):
#         a_t_minus_1_p = ancestor_matrix[p-1, t-1]
#         if t ==1:
#             z_1_t_minus_1_a_t_minus_1_p = Z_matrix[int(a_t_minus_1_p)-1, 0] #ok
#             z_1_t_minus_1_a_t_minus_1_p = np.array([z_1_t_minus_1_a_t_minus_1_p]) #ok
#         else:
#             z_1_t_minus_1_a_t_minus_1_p = Z_matrix[int(a_t_minus_1_p)-1, 1:(t-1)+1]
        
#         alpha_t_p = compute_alpha_normalized(t=t,
#                                              z_1_t_minus_1=z_1_t_minus_1_a_t_minus_1_p,
#                                              num_topics=NUM_TOPICS,
#                                              num_voc=NUM_WORDS,
#                                              lstm=lstm,
#                                              ssm=ssm)
#         alpha_t_p = alpha_t_p[x[t-1]-1]
#         alpha_matrix[p-1,t] = alpha_t_p

# alpha_T=alpha_matrix[:,-1]
# alpha_T = alpha_T / np.sum(alpha_T+1e-6)
# r = np.argmax(alpha_T)+1
# a_T_r = ancestor_matrix[int(r)-1, -1]
# z_1_T = Z_matrix[int(a_T_r)-1, 1:]